# Análisis y Modelado de Series de Tiempo de Combustibles en Guatemala: Consumo, Importación y Precios (2000–2025)

## Carga, limpieza y preparación de datos

1. Unificación de archivos
2. Selección de variables objetivo
3. Operaciones para unificar columnas
4. Estandarización de formatos (datetime y unidad en precios GQT/GAL)
5. Implementación de *caché* para leer los archivos de salida una vez que ya esten generados los posea el usuario


En caso de querer hacer el procedimiento de limpieza y unificación de datos se puede borrar el archivo `cache/df_cache.txt` este controla el proceso para hacer la unificación del proceso o leer unicamente los csv generados; si se desea usar los csv que ya se tienen estos siempre leeran los archivos de **data-ouput** y tranformando la columna "Fecha" a `DateTime` por lo que no es necesario validar, adicional el encabezado manejado, variables, control se encuentra en la sección de **Variables - Constantes**.

### Librerias

In [292]:
import pandas as pd
import numpy as np
import os

### Variables - Constantes

In [293]:
DATA_DIR = "./data-input"
OUTPUT_DIR = "./data-output"
REPORTES = "./reportes"
CACHE = "./cache"

# input files
CONSUMO_2024_XLSX = f"{DATA_DIR}/CONSUMO-HIDROCARBUROS-2024-12.xlsx"
CONSUMO_2025_XLSX = f"{DATA_DIR}/VENTAS-HIDROCARBUROS-2025-05.xlsx"
IMPORT_2024_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx"
IMPORT_2025_XLSX = f"{DATA_DIR}/IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx"
PRECIOS_2025_XLSX = f"{DATA_DIR}/Precios-Promedio-Nacionales-Diarios-2025-1.xlsx"

# output files
CONSUMO_CSV = f"{OUTPUT_DIR}/consumo_combustibles.csv"
IMPORT_CSV = f"{OUTPUT_DIR}/importacion_combustibles.csv"
PRECIOS_CSV = f"{OUTPUT_DIR}/precios_diarios.csv"
TARGET_COLUMNS = ["fecha", "regular", "superior", "diesel", "glp"]

# var control
SAVE_CONSUMO_CSV = True
SAVE_IMPORT_CSV = True
SAVE_PRECIOS_CSV = True
SAVE_REPORTES = True

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(REPORTES, exist_ok=True)
os.makedirs(CACHE, exist_ok=True)

### Carga y conversión de archivos Excel a CSV

In [294]:
def loadAndProcessFuelData(file2024, file2025, saveCsv=False, outputCsvPath=None):
    # Load 2024 data
    df2024 = pd.read_excel(file2024, skiprows=6)
    df2024.columns = df2024.columns.str.strip().str.lower()
    print(f"2024 Data: {df2024.shape[0]} rows, {df2024.shape[1]} columns")

    # Drop last 3 rows
    df2024 = df2024.iloc[:-3]
    print("Removed unnecessary rows from 2024 data.")
    print(f"Updated 2024 Data: {df2024.shape[0]} rows, {df2024.shape[1]} columns")

    # Select and process columns
    target_col_2024 = [
        "fecha",
        "gas licuado de petróleo",
        "gasolina regular",
        "gasolina superior",
        "diesel alto azufre",
        "diesel bajo azufre"
    ]
    df2024 = df2024[target_col_2024]
    df2024["diesel bajo azufre"] = df2024["diesel bajo azufre"] + df2024["diesel alto azufre"]
    df2024 = df2024.drop(columns=["diesel alto azufre"])
    print("Processed 2024 columns.")

    # Load 2025 data
    df2025 = pd.read_excel(file2025, skiprows=6)
    df2025.columns = df2025.columns.str.strip().str.lower()
    print(f"2025 Data: {df2025.shape[0]} rows, {df2025.shape[1]} columns")

    # Drop last 3 rows
    df2025 = df2025.iloc[:-3]
    print("Removed unnecessary rows from 2025 data.")
    print(f"Updated 2025 Data: {df2025.shape[0]} rows, {df2025.shape[1]} columns")

    # Select target columns
    target_col_2025 = [
        "fecha",
        "gas licuado de petróleo",
        "gasolina regular",
        "gasolina superior",
        "diesel bajo azufre"
    ]
    df2025 = df2025[target_col_2025]
    print("Processed 2025 columns.")

    # Concatenate both years
    dfResult = pd.concat([df2024, df2025], ignore_index=True)
    print("Merged 2024 and 2025 data.")
    
    # Convert 'fecha' to datetime
    dfResult["fecha"] = pd.to_datetime(dfResult["fecha"])
    print("Converted 'fecha' column to datetime.")

    # Rename and reorder columns
    dfResult.rename(columns={
        "fecha": "fecha",
        "gasolina regular": "regular",
        "gasolina superior": "superior",
        "diesel bajo azufre": "diesel",
        "gas licuado de petróleo": "glp"
    }, inplace=True)

    dfResult = dfResult[TARGET_COLUMNS]

    # Save to CSV if requested
    if saveCsv and outputCsvPath:
        dfResult.to_csv(outputCsvPath, index=False)
        print(f"Data saved to {outputCsvPath}")

    return dfResult

In [295]:
def loadAndProcessMultipleSheets(configList, saveCsv=False, outputCsvPath=None):
    dfs = []

    for config in configList:
        filePath = config["filePath"]
        sheetName = config["sheetName"]
        skipRows = config["skipRows"]
        rowsToRemove = config["rowsToRemove"]

        # Leer hoja
        df = pd.read_excel(filePath, header=[0, 1], skiprows=skipRows, sheet_name=sheetName)
        print(f"Loaded sheet '{sheetName}': {df.shape[0]} rows")

        # Eliminar filas finales
        df = df.iloc[:-rowsToRemove]

        # Seleccionar columnas objetivo
        target_columns_multi = [
            ('FECHA', 'Unnamed: 0_level_1'),
            ('Regular', 'GTQ/GALON'),
            ('Superior', 'GTQ/GALON'),
            ('Diesel', 'GTQ/GALON'),
            ('Glp Cilindro 25Lbs.', 'GTQ/LB')
        ]
        df = df[target_columns_multi]

        # Aplanar nombres
        df.columns = [f"{col1.strip()}".lower() for col1, col2 in df.columns]

        # GLP conversión de GTQ/lb a GTQ/galón (25 lbs * 4.24)
        if 'glp cilindro 25lbs.' in df.columns:
            df['glp'] = df['glp cilindro 25lbs.'] * 4.24
            df.drop(columns=['glp cilindro 25lbs.'], inplace=True)
        else:
            df['glp'] = np.nan

        # Reordenar columnas finales
        df = df[TARGET_COLUMNS]

        print(f"Processed sheet '{sheetName}' with {df.shape[0]} rows")
        dfs.append(df)

    # Concatenar todo
    df_final = pd.concat(dfs, ignore_index=True)

    # Convertir y ordenar por fecha
    df_final["fecha"] = pd.to_datetime(df_final["fecha"])
    df_final = df_final.sort_values("fecha").reset_index(drop=True)
    print("Converted and sorted 'fecha' column.")

    # Guardar si aplica
    if saveCsv and outputCsvPath:
        df_final.to_csv(outputCsvPath, index=False)
        print(f"Final data saved to {outputCsvPath}")

    return df_final

In [296]:
def loadAllData():
    CACHE_FLAG = os.path.join(CACHE, "df_cache.txt")
    if not os.path.exists(CACHE_FLAG):
        configList = [
            {"filePath": PRECIOS_2025_XLSX, "sheetName": 0, "skipRows": 7, "rowsToRemove": 9},
            {"filePath": PRECIOS_2025_XLSX, "sheetName": 1, "skipRows": 7, "rowsToRemove": 2},
            {"filePath": PRECIOS_2025_XLSX, "sheetName": 2, "skipRows": 7, "rowsToRemove": 2},
            {"filePath": PRECIOS_2025_XLSX, "sheetName": 3, "skipRows": 6, "rowsToRemove": 2},
            {"filePath": PRECIOS_2025_XLSX, "sheetName": 4, "skipRows": 6, "rowsToRemove": 2}
        ]

        df_consumo = loadAndProcessFuelData(CONSUMO_2024_XLSX, CONSUMO_2025_XLSX, saveCsv=SAVE_CONSUMO_CSV, outputCsvPath=CONSUMO_CSV)
        df_importaciones = loadAndProcessFuelData(IMPORT_2024_XLSX, IMPORT_2025_XLSX, saveCsv=SAVE_IMPORT_CSV, outputCsvPath=IMPORT_CSV)

        df_precios = loadAndProcessMultipleSheets(
            configList=configList,
            saveCsv=SAVE_PRECIOS_CSV,
            outputCsvPath=PRECIOS_CSV
        )
        
        with open(CACHE_FLAG, "w") as f:
            f.write("cached")

        print("Procesamiento completado desde archivos fuente.")
    else:
        df_consumo = pd.read_csv(CONSUMO_CSV, parse_dates=["fecha"])
        df_importaciones = pd.read_csv(IMPORT_CSV, parse_dates=["fecha"])
        df_precios = pd.read_csv(PRECIOS_CSV, parse_dates=["fecha"])

        print("Datos cargados desde cache CSV.")
    return df_consumo, df_importaciones, df_precios

### Generación de reportes para explorar data

In [297]:
def generateBasicReport(df, name):
    report_path = os.path.join(REPORTES, f"{name}_reporte.txt")

    with open(report_path, "w", encoding="utf-8") as f:
        f.write("REPORTE BÁSICO DE DATAFRAME\n")
        f.write("=" * 40 + "\n\n")

        # Dimensiones
        f.write(f"Total filas: {df.shape[0]}\n")
        f.write(f"Total columnas: {df.shape[1]}\n\n")

        # Columnas
        f.write("Columnas:\n")
        for col in df.columns:
            f.write(f"  - {col}\n")
        f.write("\n")

        # Tipos de datos
        f.write("Tipos de datos por columna:\n")
        f.write(df.dtypes.to_string())
        f.write("\n\n")

        # Nulos
        f.write("Valores nulos por columna:\n")
        f.write(df.isnull().sum().to_string())
        f.write("\n\n")

        # Porcentaje de nulos
        f.write("Porcentaje de nulos por columna:\n")
        null_pct = (df.isnull().mean() * 100).round(2)
        f.write(null_pct.to_string())
        f.write("\n\n")

        # Duplicados
        f.write(f"Total filas duplicadas: {df.duplicated().sum()}\n\n")

        # Estadísticas básicas (numéricas)
        f.write("Estadísticas básicas (solo numéricas):\n")
        f.write(df.describe().to_string())
        f.write("\n\n")

        # Medianas
        f.write("Mediana por columna numérica:\n")
        f.write(df.median(numeric_only=True).to_string())
        f.write("\n")

    print(f"Reporte generado en: {report_path}")


In [298]:
if SAVE_REPORTES:
  df_consumo, df_importaciones, df_precios = loadAllData()
  generateBasicReport(df_consumo, "consumo_combustibles")
  generateBasicReport(df_importaciones, "importacion_combustibles")
  generateBasicReport(df_precios, "precios_combustibles")

Datos cargados desde cache CSV.
Reporte generado en: ./reportes\consumo_combustibles_reporte.txt
Reporte generado en: ./reportes\importacion_combustibles_reporte.txt
Reporte generado en: ./reportes\precios_combustibles_reporte.txt


## Análisis exploratorio

### Librerias

In [299]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

### Variables - Constantes

In [300]:
# Variables control para mostrar histogramas, series de tiempo, heatmaps y casos especiales (pandemia y reciente)
SHOW_ST = False
SHOW_HT = False
SHOW_HM = False
SHOW_SC = False

IMAGES_DIR = "./images"
HISTOGRAMA = os.path.join(IMAGES_DIR, "histogramas")
ST = os.path.join(IMAGES_DIR, "series_tiempo")
HM = os.path.join(IMAGES_DIR, "heatmap")
SC = os.path.join(IMAGES_DIR, "special_cases")

os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(HISTOGRAMA, exist_ok=True)
os.makedirs(ST, exist_ok=True)
os.makedirs(HM, exist_ok=True)
os.makedirs(SC, exist_ok=True)

### Preparación previa

In [301]:
df_consumo, df_importaciones, df_precios = loadAllData()
df_consumo.set_index('fecha', inplace=True)
df_importaciones.set_index('fecha', inplace=True)
df_precios.set_index('fecha', inplace=True)

Datos cargados desde cache CSV.


### Resumen Estadístico

In [302]:
print("Resumen consumo:\n", df_consumo.describe())
print("\nResumen importaciones:\n", df_importaciones.describe())
print("\nResumen precios:\n", df_precios.describe())

Resumen consumo:
              regular       superior        diesel            glp
count     305.000000     305.000000  3.050000e+02     305.000000
mean   405016.802384  474465.500605  8.801976e+05  322885.822617
std    225100.212304  117354.116627  2.319830e+05  118981.768608
min    160741.900000  300242.781667  5.076627e+05  167818.137897
25%    216560.790000  385044.965833  7.012555e+05  227154.969762
50%    290286.650000  424438.710000  8.076921e+05  268875.095238
75%    570267.250000  575720.920000  1.062409e+06  407953.780000
max    942394.340000  790948.040000  1.474651e+06  600453.870000

Resumen importaciones:
             regular      superior        diesel           glp
count  2.930000e+02  2.930000e+02  2.930000e+02  2.930000e+02
mean   4.199959e+05  4.945875e+05  8.995610e+05  4.222037e+05
std    2.471180e+05  1.578443e+05  2.945229e+05  2.059063e+05
min    8.101530e+04  1.702925e+05  2.297647e+05  1.005615e+05
25%    2.087960e+05  3.724865e+05  6.931605e+05  2.305846e+05


### Histogramas

In [303]:
for df, nombre in zip([df_consumo, df_importaciones, df_precios], 
                      ['consumo', 'importaciones', 'precios']):
    axes = df.hist(figsize=(10, 6), bins=30)

    for ax in axes.flatten():
        ax.tick_params(axis='both', labelsize=8)
        ax.xaxis.set_major_formatter(mtick.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(style='scientific', axis='x', scilimits=(3, 5))
        ax.ticklabel_format(style='scientific', axis='y', scilimits=(3, 5))
        ax.set_title(ax.get_title(), fontsize=9)
        ax.set_xlabel("Valor", fontsize=9)
        ax.set_ylabel("Frecuencia", fontsize=9)

    plt.suptitle(f'Distribución de variables - {nombre.capitalize()}', fontsize=12)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    path_salida = os.path.join(HISTOGRAMA, f"{nombre}_histograma.png")
    plt.savefig(path_salida, dpi=150)
    if SHOW_HT:
        plt.show()
    else:
        plt.close()

### Series de tiempo

In [304]:
def plotST(df, titulo, nombre_archivo_base):
    subdir = os.path.join(ST, nombre_archivo_base)
    os.makedirs(subdir, exist_ok=True)

    # Gráfica combinada
    df.plot(figsize=(12, 6))
    plt.title(f"Series de tiempo - {titulo}", fontsize=12)
    plt.xlabel("Fecha", fontsize=10)
    plt.ylabel("Valor", fontsize=10)
    plt.grid(True)
    plt.tight_layout()

    # Guardar imagen combinada
    path = os.path.join(subdir, f"{nombre_archivo_base}_combinada.png")
    plt.savefig(path, dpi=150)
    if SHOW_ST:
      plt.show()
    else:
      plt.close()

    # Gráficas individuales por columna
    for col in df.columns:
        plt.figure(figsize=(10, 5))
        df[col].plot()
        plt.title(f"{titulo} - {col.capitalize()}", fontsize=11)
        plt.xlabel("Fecha", fontsize=9)
        plt.ylabel("Valor", fontsize=9)
        plt.grid(True)
        plt.tight_layout()

        # Guardar cada gráfica individual
        col_sanitized = col.lower().replace(" ", "_")
        path_indiv = os.path.join(subdir, f"{nombre_archivo_base}_{col_sanitized}.png")
        plt.savefig(path_indiv, dpi=150)
        if SHOW_ST:
          plt.show()
        else:
          plt.close()

In [305]:
plotST(df_consumo, "Consumo", "consumo")
plotST(df_importaciones, "Importaciones", "importaciones")
plotST(
    df_precios.resample('ME').mean(),
    "Precios (promedios mensuales)",
    "precios"
)

### HeatMap mensual por año

In [306]:
def heatMapMonthXYear(df, titulo, nombre_archivo_base):
    subdir = os.path.join(HM, nombre_archivo_base)
    os.makedirs(subdir, exist_ok=True)

    df_cp = df.copy()
    df_cp['año'] = df_cp.index.year
    df_cp['mes'] = df_cp.index.month

    for col in df_cp.columns.drop(['año', 'mes'], errors='ignore'):
        tabla = df_cp.pivot_table(index='año', columns='mes', values=col, aggfunc='sum')
        plt.figure(figsize=(10, 6))
        sns.heatmap(tabla, cmap="YlGnBu", annot=False, fmt=".0f")
        plt.title(f"{titulo} - {col}")
        plt.xlabel("Mes")
        plt.ylabel("Año")
        plt.tight_layout()

        # Guardar imagen
        col_sanitized = col.lower().replace(" ", "_")
        path = os.path.join(subdir, f"{nombre_archivo_base}_{col_sanitized}.png")
        plt.savefig(path, dpi=150)

        if SHOW_HM:
            plt.show()
        else:
            plt.close()

In [307]:
heatMapMonthXYear(df_importaciones, "Importaciones mensuales", "importaciones")
heatMapMonthXYear(df_consumo, "Consumo mensual", "consumo")
heatMapMonthXYear(df_precios, "Precio mensual", "precios")


### Comportamiento en pandemia (2020-2021)

In [308]:
def plotYearsPandemic(df, titulo, nombre_archivo_base):
    pandemia_dir = os.path.join(SC, "pandemia_2020-2021")
    subdir = os.path.join(pandemia_dir, nombre_archivo_base)

    os.makedirs(subdir, exist_ok=True)

    # Subconjunto de datos
    df_pandemia = df.loc['2020':'2021']

    # Gráfico combinado
    df_pandemia.plot(figsize=(12, 6))
    plt.title(f"{titulo} durante la pandemia (2020-2021)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(subdir, f"{nombre_archivo_base}_combinado.png"), dpi=150)
    if SHOW_SC:
        plt.show()
    else:
        plt.close()

    # Gráficos individuales
    for col in df_pandemia.columns:
        plt.figure(figsize=(10, 5))
        df_pandemia[col].plot()
        plt.title(f"{titulo} - {col} (2020-2021)")
        plt.grid(True)
        plt.tight_layout()
        col_sanitized = col.lower().replace(" ", "_")
        plt.savefig(os.path.join(subdir, f"{nombre_archivo_base}_{col_sanitized}.png"), dpi=150)
        if SHOW_SC:
            plt.show()
        else:
            plt.close()


In [309]:
plotYearsPandemic(df_consumo, "Consumo", "consumo")
plotYearsPandemic(df_importaciones, "Importaciones", "importaciones")
plotYearsPandemic(df_precios, "Precios", "precios")

### Comportamiento reciente (2022 - 2025)

In [310]:
def plotYearRange(df, titulo, nombre_archivo_base, anio_inicio, anio_fin=None):
    # Validación de índice tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        print("El índice del DataFrame no es datetime. No se puede filtrar por año.")
        return

    # Validar si el año inicio existe
    años_disponibles = df.index.year.unique()
    if anio_inicio not in años_disponibles:
        print(f"Año de inicio {anio_inicio} no existe en los datos.")
        return

    # Si no se proporciona año final, usar el último año disponible
    if anio_fin is None:
        anio_fin = df.index.year.max()

    if anio_fin < anio_inicio:
        print(f"Año final {anio_fin} no puede ser menor que el año inicial {anio_inicio}.")
        return

    # Filtrar el DataFrame
    df_rango = df.loc[f'{anio_inicio}':f'{anio_fin}']
    if df_rango.empty:
        print(f"No hay datos en el rango {anio_inicio} - {anio_fin}.")
        return

    # Crear carpeta destino
    rango_str = f"{anio_inicio}_{anio_fin}"
    subdir = os.path.join(SC, f"periodo_{rango_str}", nombre_archivo_base)
    os.makedirs(subdir, exist_ok=True)

    # Gráfico combinado
    df_rango.plot(figsize=(12, 6))
    plt.title(f"{titulo} ({anio_inicio}-{anio_fin})")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(subdir, f"{nombre_archivo_base}_combinado.png"), dpi=150)
    if SHOW_SC:
        plt.show()
    else:
        plt.close()

    # Gráficos individuales
    for col in df_rango.columns:
        plt.figure(figsize=(10, 5))
        df_rango[col].plot()
        plt.title(f"{titulo} - {col} ({anio_inicio}-{anio_fin})")
        plt.grid(True)
        plt.tight_layout()
        col_sanitized = col.lower().replace(" ", "_")
        plt.savefig(os.path.join(subdir, f"{nombre_archivo_base}_{col_sanitized}.png"), dpi=150)
        if SHOW_SC:
            plt.show()
        else:
            plt.close()

In [311]:
plotYearRange(df_consumo, "Consumo", "consumo", 2022)
plotYearRange(df_importaciones, "Importaciones", "importaciones", 2022)
plotYearRange(df_precios, "Precios", "precios", 2022)

## Otra sección

In [312]:
# Para más tarde xd

# from datetime import datetime

# import matplotlib.pyplot as plt

# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from statsmodels.tsa.arima.model import ARIMA

# import pmdarima as pm  # auto_arima

# from prophet import Prophet

# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.neural_network import MLPRegressor  # NN simple o usar tensorflow dependerá por ahora el simple


# serie_precio_gasolina_regular      # Series de tiempo de precios mensual (Ciudad Capital)
# serie_import_diesel_total          # Serie mensual de importación total de diésel
# serie_consumo_gas_licuado          # Serie mensual de consumo de gas propano